In [1]:
from PIL import Image
import os.path
import numpy as np
import itertools

In [116]:
def resizeImg(action, x , y):
    avgX = x
    avgY = y
    folder = ['PNEUMONIA','NORMAL']
    for foldername in folder:
        filelist= [file for file in os.listdir('chest_xray/' + action + '/' + foldername + '/') if file.endswith('.jpeg')]
        for filename in filelist:
            img = Image.open('chest_xray/' + action + '/' + foldername + '/' + filename)
            imgResize = img.resize((avgX, avgY), Image.LANCZOS)
            imgResize.save('chest_xray/' + action + '/resized250x250/' + foldername + '/' + filename)
            img.close()

In [117]:
resizeImg('test',250,250)

In [ ]:
resizeImg('train',250,250)

In [6]:
def convertImg(action):
    #convert all RGB files to grey scale. RGB is 3d and greyscale is 2d
    folder = ['PNEUMONIA','NORMAL']
    for foldername in folder:
        filelist= [file for file in os.listdir('chest_xray/' + action + '/resized250x250/' + foldername + '/') if file.endswith('.jpeg')]
        for filename in filelist:
        #for filename in itertools.islice(filelist, 0, 50):
            img = Image.open('chest_xray/' + action + '/resized250x250/' + foldername + '/' + filename)
            imgData = list(img.getdata())
            imgData = np.asarray(imgData)
            #print(imgData.shape)
            #print(imgData.ndim)
            #x = imgData.reshape(150,150)
            #The array is 1d before reshaping for grey scale images and 2d for RGB. We find all the RGB images and
            #convert them to grey scale and save with prefix 'gs_'
            if imgData.ndim == 2:
                img_gs = Image.open('chest_xray/' + action + '/resized250x250/' + foldername + '/' + filename).convert('L')
                img_gs.save('chest_xray/' + action + '/resized250x250/' + foldername + '/gs_'+ filename)
                img_gsData = list(img_gs.getdata())
                img_gsData = np.asarray(img_gsData)
                #print(filename)
                #print(img_gsData.ndim)

In [ ]:
convertImg('train')

In [ ]:
convertImg('test')

In [107]:
def process_image(action):
    X = []
    y = []
    folder = ['PNEUMONIA','NORMAL']

    for foldername in folder:
        filelist= [file for file in os.listdir('chest_xray/' + action + '/resized250x250/' + foldername + '/') if file.endswith('.jpeg')]
        for filename in filelist:
        #for filename in itertools.islice(filelist, 0, 5):
            #print(filename)
            img = Image.open('chest_xray/' + action + '/resized250x250/' + foldername + '/' + filename)
            #imgData = list(img.getdata())
            #imgData = np.asarray(imgData)
            #imgData = img.getdata()
            #imgData = np.asarray(imgData)
            imgData = np.asarray(img)
            #print(imgData.shape)
            #print(imgData.ndim)
            if imgData.ndim == 2:
                x = imgData.reshape(250,250)
                X.append(x)
                if foldername == 'NORMAL':
                    y.append(0)
                else:
                    y.append(1)
            img.close()
    return X, y

In [108]:
X_train, y_train = process_image('train')

In [109]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
print("shape of X_train: " + str(X_train.shape))
print("length of X_train: " + str(len(X_train)))
print("shape of y_train: " + str(y_train.shape))
print("length of y_train: " + str(len(y_train)))
print("value of y_train[0]: " + str(y_train[0]))

shape of X_train: (5216, 250, 250)
length of X_train: 5216
shape of y_train: (5216,)
length of y_train: 5216
value of y_train[0]: 1


In [110]:
img = Image.fromarray(X_train[1], 'L')
img.show()

In [111]:
y_train[1]

1

In [121]:
X_test, y_test = process_image('test')

In [122]:
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)
print("shape of X_test: " + str(X_test.shape))
print("length of X_test: " + str(len(X_test)))
print("shape of y_test: " + str(y_test.shape))
print("length of y_test: " + str(len(y_test)))
print("value of y_test[0]: " + str(y_test[0]))

shape of X_test: (624, 250, 250)
length of X_test: 624
shape of y_test: (624,)
length of y_test: 624
value of y_test[0]: 1


In [123]:
img = Image.fromarray(X_test[1], 'L')
img.show()

In [124]:
y_test[1]

1

In [125]:
from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [142]:
from keras.callbacks import ReduceLROnPlateau , ModelCheckpoint , LearningRateScheduler
lr_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.1, epsilon=0.0001, patience=1, verbose=1)

In [143]:
filepath="transferlearning_weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [135]:
from keras.models import Sequential , Model
from keras.layers import Dense , Activation
from keras.layers import Dropout , GlobalAveragePooling2D
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD , RMSprop , Adadelta , Adam
from keras.layers import Conv2D , BatchNormalization
from keras.layers import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [133]:
from keras.applications.inception_v3 import InceptionV3
# create the base pre-trained model
base_model = InceptionV3(weights=None, include_top=False , input_shape=(250, 250, 1))

In [136]:
x = base_model.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
predictions = Dense(2, activation='sigmoid')(x)

In [137]:
model = Model(inputs=base_model.input, outputs=predictions)

In [138]:
model.compile(loss='categorical_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])

In [139]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 250, 250, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 124, 124, 32) 288         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 124, 124, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 124, 124, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [140]:
batch_size = 64
epochs = 10

In [144]:
history = model.fit(X_train, y_train, validation_data = (X_test , y_test),callbacks=[lr_reduce,checkpoint] ,
          epochs=epochs)

ValueError: Error when checking input: expected input_1 to have 4 dimensions, but got array with shape (5216, 250, 250)